In [33]:
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH

In [39]:
df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '종로구_카페 정보.csv'), sep=',')

In [40]:
df['dong'] = df.iloc[:, 3]
print(df['dong'])

0      서울 종로구 종로2가 84-8
1          서울 종로구 재동 86
2       서울 종로구 계동 146-1
3         서울 종로구 소격동 86
4       서울 종로구 필운동 32-1
5      서울 종로구 효자동 164-7
6      서울 종로구 관훈동 155-2
7       서울 종로구 부암동 97-5
8        서울 종로구 청진동 246
9     서울 종로구 삼청동 35-169
10        서울 종로구 중학동 19
11     서울 종로구 원서동 221-1
12      서울 종로구 재동 84-14
13       서울 종로구 내자동 196
14     서울 종로구 명륜4가 94-2
15     서울 종로구 신문로1가 115
16     서울 종로구 통의동 35-21
17      서울 종로구 신문로1가 25
18     서울 종로구 이화동 9-443
19       서울 종로구 익선동 144
20     서울 종로구 부암동 254-2
21    서울 종로구 명륜4가 154-2
22        서울 종로구 안국동 83
23     서울 종로구 사직동 262-4
24        서울 종로구 누하동 22
25        서울 종로구 돈의동 13
26        서울 종로구 통의동 10
27     서울 종로구 삼청동 28-21
28      서울 종로구 삼청동 15-2
29       서울 종로구 내자동 142
Name: dong, dtype: object


In [41]:
df['dong'] = df['dong'].str.split(' ').str[1]
print(df['dong'])

0     종로구
1     종로구
2     종로구
3     종로구
4     종로구
5     종로구
6     종로구
7     종로구
8     종로구
9     종로구
10    종로구
11    종로구
12    종로구
13    종로구
14    종로구
15    종로구
16    종로구
17    종로구
18    종로구
19    종로구
20    종로구
21    종로구
22    종로구
23    종로구
24    종로구
25    종로구
26    종로구
27    종로구
28    종로구
29    종로구
Name: dong, dtype: object


In [45]:
df['naver_keyword'] = df['dong'].iloc[0,]
print(df['naver_keyword'])

0     종로구
1     종로구
2     종로구
3     종로구
4     종로구
5     종로구
6     종로구
7     종로구
8     종로구
9     종로구
10    종로구
11    종로구
12    종로구
13    종로구
14    종로구
15    종로구
16    종로구
17    종로구
18    종로구
19    종로구
20    종로구
21    종로구
22    종로구
23    종로구
24    종로구
25    종로구
26    종로구
27    종로구
28    종로구
29    종로구
Name: naver_keyword, dtype: object
